In [2]:
from bs4 import BeautifulSoup
import requests
from lxml import html  
import csv
import requests
from time import sleep
import re
import argparse
import sys
import pandas as pd
import time as t
import sys
import numpy as np

In [5]:
headers = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36'}
links_with_text = []
final_city_links =[]
info_scraped = {}

#scraps all urls on the page
def parse_url(url) :
	response=requests.get(url,headers=headers)
	soup=BeautifulSoup(response.content,'lxml')
	t.sleep(3)

	for a in soup.find_all('a', href=True, class_ = 'css-166la90'): 
    		if a.text: 
        		links_with_text.append(a['href'])

#save only business URL
def clean_urls(links_with_text):
	for link in links_with_text:
		if (link[0:5] =="/biz/"):
			info_scraped['URL'] = "https://www.yelp.com"+link
			final_city_links.append(info_scraped['URL'])
	print(final_city_links)		
	df = pd.DataFrame({'URL':final_city_links})
	return(df)
						
#main function takes in list of page numbers as input and scraps it		
if __name__=="__main__":
	argparser = argparse.ArgumentParser()
	argparser.add_argument('page_no_file')
	argparser.parse_args()
	filename= sys.argv[1]
	page_no = np.loadtxt(filename, delimiter=',')
	for m in page_no:
		yelp_url  = "https://www.yelp.com/search?cflt=restaurants&find_loc=Chicago&start=%s"%(m)
		print(m)
		scraped_data = parse_url(yelp_url)
	final_links = clean_urls(links_with_text)
	final_links.to_csv("url_yelp.csv")

usage: ipykernel_launcher.py [-h] page_no_file
ipykernel_launcher.py: error: the following arguments are required: page_no_file


SystemExit: 2

In [4]:
def res_scraper(url):
	driver = webdriver.Firefox(options=fireFoxOptions)
	driver.get(url)
	
	t.sleep(1)
	page = driver.page_source
	soup = BeautifulSoup(page, 'lxml')
	soup2 = BeautifulSoup(page, 'html.parser')
	final_data = []

	# retrieve the total page number
	info_scraped = {}
	final_tag = ''
	final_address = ''

	info_scraped['restaurant_name'] = None
	info_scraped['restaurant_url'] = url
	info_scraped['restaurant_tag'] = None
	info_scraped['restaurant_neighborhood'] = None
	info_scraped['restaurant_address'] = None
	info_scraped['ratings'] = None
	info_scraped['review_number'] = None
	info_scraped['price'] = None

	all = soup.find('div', {'class': "main-content-wrap main-content-wrap--full"})
	special_divs = soup2.find_all('div',{'class':'main-content-wrap'})
	# retrieve tags and append to one string
	try:
		for text in special_divs:
    		tags = text.find_all('a', href = re.compile('/c/'))
		
		for tag in tags:
			final_tag += tag.text + ','
			info_scraped['restaurant_tag'] = final_tag
      		
	except:
		print (None)
		
    # retrieve restaurant name
	try:
		info_scraped['restaurant_name'] = all.find('h1').text
	except:
		print(None)

    # retrieve neighborhood on yelp, which now is CT_ID_10
	try:
		for text in special_divs:
    			neighbor = text.find_all('p', {'class': 'css-8yg8ez'})
		info_scraped['restaurant_neighborhood'] = neighbor[0].text
	except:
		print(None)

    # retrieve address and append road, city, zip code to one string
	try:
		addresses = all.find('address').find('p').find_all('span',{'class': 'raw__373c0__3rcx7'})
		addresses2 = all.find('address').find('p',{'class':'css-znumc2'}).find_all('span',{'class': 'raw__373c0__3rcx7'})
		
		for address in addresses:
            		final_address += address.text + ','
		for address in addresses2:
            		final_address += address.text + ','
		info_scraped['restaurant_address'] = final_address
	except:
		print(None)

    # retrieve the average rating of each restaurant
	try:
		info_scraped['ratings'] = all.find('div', {'aria-label': re.compile(' star rating')})['aria-label']
	except:
		print(None)

    # retrieve total review numbers
	try:
		review_number = all.find('span', {'class': 'css-bq71j2'}).text
		review_number = [int(i) for i in review_number.split() if i.isdigit()][0]
		info_scraped['review_number'] = review_number
	except:
		print(None)

    # retrieve price category listed on YELP
	try:
		price_data = driver.find_element_by_xpath('/html/body/div[2]/div[3]/yelp-react-root/div/div[2]/div[1]/div[1]/div/div/span[2]/span').text
		if price_data[0] == '$':
			info_scraped['price'] = price_data
		else:
			info_scraped['price'] = ''
	except:
		print(None)

	final_data.append(info_scraped)

	df = pd.DataFrame(final_data)
	df.index += 1
	driver.quit()

	return df


iteration_from = 60
iteration_end = 61
#iteration_end = len(urls)
review_data = []

# set driver to headless mode
fireFoxOptions = webdriver.FirefoxOptions()
fireFoxOptions.set_headless()

for i in range(iteration_from, iteration_end):
	print(str(datetime.now()) + " "+ str(i) + " restaurant out of " + str(len(urls)))
	item = urls[i] + '?sort_by=date_desc'
	resreview = res_scraper(item)
	review_data.append(resreview)
	review_all = pd.concat(review_data)
    
# encoding is utf-8-sig
review_all.to_csv("Res_info60-61.csv", encoding='utf-8-sig')

TabError: inconsistent use of tabs and spaces in indentation (1431811388.py, line 30)

In [ ]:
#using random delay for time.sleep()
delays = [7, 4, 6, 2, 10, 19]
delay = np.random.choice(delays)

def get_review(url, res_name, res_address):
    binary = FirefoxBinary('/usr/bin/firefox')
    opts = webdriver.FirefoxOptions()
    opts.add_argument("--headless")
    driver = webdriver.Firefox(firefox_binary=binary, firefox_options=opts )
    driver.get(url)
    
    time.sleep(delay)
    page = driver.page_source
    soup = BeautifulSoup(page, 'lxml')
    review_num = 0
    final_data = []
    num_page = 1
    info_scraped = {}
    info_scraped['reviewer_name'] = None
    #info_scraped['reviewer_stat'] = None
    info_scraped['reviewer_friends'] = None
    info_scraped['reviewer_reviews'] = None
    info_scraped['reviewer_photos'] = None
    info_scraped['ratings'] = None
    info_scraped['comment'] = None
    info_scraped['review_date'] = None
    info_scraped['reviewer_origin'] = None
    info_scraped['reviewer_profile'] = None
    # retrieve the total page number, if there is no information about this, it means the reviews have less than one full page, set page number to 1.
    try:
        total_page = driver.find_element_by_xpath('/html/body/div[2]/div[3]/yelp-react-root/div/div[3]/div/div/div[2]/div/div[1]/div[2]/section[2]/div[2]/div/div[4]/div[2]/span').text
        print(total_page)
        totalpage = [int(s) for s in total_page.split() if s.isdigit()]

        num_page = totalpage[-1]
        print(num_page)
        #num_page = 1

    except:
        print(None)

    # iterate through all pages
    
    print(url)
    for page_np in range(num_page):
        print('[{}] {} scraped page out of {}'.format(datetime.now(), page_np, num_page))
        time.sleep(2)
        page = driver.page_source
        #soup = BeautifulSoup(page, 'lxml')
        soup2 = BeautifulSoup(page, 'lxml')
        
        # retrieve all data on the site
        all = soup.find_all('div', {'class': "main-content-wrap main-content-wrap--full"})
 
        #special_all_stat = soup2.find_all('div',{'class': " margin-t0-5__373c0__1VMSL border-color--default__373c0__3-ifU"})
        special_all_reviews = soup2.find_all('div',{'class': "review__373c0__13kpL border-color--default__373c0__3-ifU"})
        review_num += len(special_all_reviews)

        for i in range(len(special_all_reviews)):
            info_scraped = {}
            default = 'https://www.yelp.com'
            stat = ''
            origin = ''
            # retrieve reviewer name
            try:
                special_user = special_all_reviews[i].find('div',{'class': "user-passport-info border-color--default__373c0__3-ifU"})
                
                info_scraped['reviewer_name'] = special_user.find('a').text
                #print(info_scraped['reviewer_name'])
            except:
                print(None)

            # retrieve reviewer statistic, like number of friends, number of reviews, elite or not.
            try:
                
                for j in special_all_reviews[i].find_all('span', {'class': 'css-1dgkz3l'}) :
                    stat += j.text
                    stat += " "
                #print(stat)
                info_scraped['reviewer_friends'] = stat.split()[0]
                info_scraped['reviewer_reviews'] = stat.split()[1]
                info_scraped['reviewer_photos'] = stat.split()[2]
            except:

                print(None)

            # retrieve the rating of this review
            try:
                
                info_scraped['ratings'] = special_all_reviews[i].find('div', {"aria-label": re.compile('star rating')})["aria-label"].split()[0]
                #print(info_scraped['ratings'])
            except:
                print(None)

            # retrieve the comment text the reviewer left
            try:
                
                info_scraped['comment'] = special_all_reviews[i].find('p', {'class': 'comment__373c0__1M-px css-n6i4z7'}).find('span', {'class': 'raw__373c0__3rcx7'}).text
               
                #print(info_scraped['comment'])
            except:

                print(None)

            # retrieve the date of review
            try:
                info_scraped['review_date'] = datetime.strptime(special_all_reviews[i].find('span', {
                    'class': 'css-e81eai'}).text,'%m/%d/%Y').date()

                #print(info_scraped['review_date'])
            except:
                print(None)

            # retrieve origin of the reviewer and append them to one string
            try:
                origin = special_all_reviews[i].find('span',{'class':'css-n6i4z7'}).text
                 
                info_scraped['reviewer_origin'] = origin
                
            except:
                print(None)

            # retrieve profile website of each reviewer, prepared to retrieve the his history of ratting record
            try:
                info_scraped['reviewer_profile'] = default + special_all_reviews[i].find('a', {'class': 'css-166la90'}).attrs['href']
               
            except:
                print(None)
            print("********************************************************************************")
            
            final_data.append(info_scraped)
            
        #find no of clickable buttons
        clickable_button = soup.find_all('div',{'class': "pagination-link-container__373c0__1mmdE border-color--default__373c0__3-ifU"})
        clicking_links = len(clickable_button)+2
        click_link = str(clicking_links)
        
        # click the next button to go to next page
            
        if page_np == num_page-1:
            break
            
        else:
            driver.find_element_by_xpath(
                '//*[@id="wrap"]/div[3]/yelp-react-root/div/div[3]/div/div/div[2]/div/div[1]/div[2]/section[2]/div[2]/div/div[4]/div[1]/div/div['+click_link+']/span/a/span').click()

    address = res_address.strip()
    restaurant_name = [res_name] * review_num
    address = [address] * review_num

    driver.quit()

    df = pd.DataFrame(final_data)

    df['restaurant name'] = pd.Series(restaurant_name)
    df['address'] = pd.Series(address)
    df.index += 1
    print(df)
    return df


iteration_from = 10
iteration_end = 20
review_data = []

for i in range(iteration_from, iteration_end):
    print(str(i) + " restaurant out of " + str(len(urls)))
    item = urls[i]
    name = res_name[i]
    address = res_add[i]
    resreview = get_review(item, name, address)
    review_data.append(resreview)
    review_all = pd.concat(review_data)
    review_all.to_csv("Reviews"+str(iteration_from)+"-"+str(iteration_end)+".csv")


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.proxy import Proxy, ProxyType

boston_res_info = pd.read_csv("combined_reviews_info13001.csv", encoding='utf-8-sig')
rev_urls = boston_res_info['reviewer_profile'].tolist()

delays = [1,2,3]
delay = np.random.choice(delays)

#keep changing proxy list based on https://sslproxies.org/
def get_review(url):

    myProxy = ['167.172.123.221','129.21.253.179','159.89.221.73']
    proxys = np.random.choice(myProxy)
    proxy = Proxy({
    'proxyType': ProxyType.MANUAL,
    'httpProxy': proxys,
    'ftpProxy': proxys,
    'sslProxy': proxys,
    })

    opts = webdriver.FirefoxOptions()
    opts.add_argument("--headless")
    
    driver = webdriver.Firefox(firefox_options=opts,proxy=proxy,executable_path='C:/Users/user/Downloads/geckodriver.exe')
    driver.get(url)
    time.sleep(delay)

    print(url)
    page = driver.page_source
    soup = BeautifulSoup(page, 'lxml')

    info_scraped = {}
    final_data = []

    # retrieve the total page number
    try:
        info_scraped['Name'] = soup.find('h1').text
    except:
        print(None)

    try:
        info_scraped['Origin'] = soup.find('h3', {'class': 'user-location alternate'}).text
    except:
        print(None)

    time.sleep(delay)
    # retrieve the number of rating history recorde
    try:
        rating_list = soup.find_all('td', {'class': 'histogram_count'})
        info_scraped['5_star'] = rating_list[0].text
        info_scraped['4_star'] = rating_list[1].text
        info_scraped['3_star'] = rating_list[2].text
        info_scraped['2_star'] = rating_list[3].text
        info_scraped['1_star'] = rating_list[4].text
    except:
        print(None)

    final_data.append(info_scraped)
    df = pd.DataFrame(final_data)
    df.index += 1

    driver.quit()
    return df


iteration_from = 500
iteration_end = 1000
review_data = []
for i in range(iteration_from, iteration_end):
    print(str(datetime.now()) + ' ' + str(i) + " reviewer out of " + str(len(rev_urls)))
    item = rev_urls[i]

    reviewer_info = get_review(item)
    review_data.append(reviewer_info)
    review_star = pd.concat(review_data)
    review_star.to_csv("Rating distribution-13001-"+str(iteration_from)+"-"+str(iteration_end)+".csv", encoding='utf-8-sig')